In [1]:
import torch
import torchvision
from torchvision import transforms
import os

In [2]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
use_cuda = torch.cuda.is_available()
use_cuda

False

In [4]:
base_dir = './01.데이터/'
train_dir = os.path.join(base_dir, 'Training/01.원천데이터/')
validation_dir = os.path.join(base_dir, 'Validation/01.원천데이터/')
train_mono_dir = os.path.join(train_dir, 'TS_01. MONO')
train_color_dir = os.path.join(train_dir, 'TS_02. COLOR')
train_archive_dir = os.path.join(train_dir, 'TS_03. 보관기간')
validation_mono_dir = os.path.join(validation_dir, 'VS_01. MONO')
validation_color_dir = os.path.join(validation_dir, 'VS_02. COLOR')
validation_archive_dir = os.path.join(validation_dir, 'VS_03. 보관기간')

In [5]:
print('total training mono images:', len(os.listdir(train_mono_dir)))

total training mono images: 55692


In [6]:
print('total training color images:', len(os.listdir(train_color_dir)))

total training color images: 43146


In [7]:
print('total training archive images:', len(os.listdir(train_archive_dir)))

total training archive images: 23045


In [8]:
print('total validation mono images:', len(os.listdir(validation_mono_dir)))

total validation mono images: 6961


In [9]:
print('total validation color images:', len(os.listdir(validation_color_dir)))

total validation color images: 5393


In [10]:
print('total validation archive images:', len(os.listdir(validation_archive_dir)))

total validation archive images: 2880


In [11]:
def checkImage(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False
    pass

In [12]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

In [13]:
train_data = torchvision.datasets.ImageFolder(root = train_dir, transform = img_transforms, is_valid_file = checkImage)
val_data = torchvision.datasets.ImageFolder(root = validation_dir, transform = img_transforms, is_valid_file = checkImage)

In [14]:
#By default, PyTorch’s data loaders are set to a batch_size of 1.
BATCH_SIZE = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE)

In [15]:
sample = next(iter(train_data_loader))
imgs, lbls = sample
print(lbls)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [16]:
imgs[0]

tensor([[[-2.0494, -2.0494, -2.0494,  ..., -2.0494, -2.0494, -2.0494],
         [-2.0665, -2.0323, -1.9980,  ..., -2.0494, -2.0494, -2.0494],
         [-2.0494, -1.9980, -1.9809,  ..., -2.0494, -2.0494, -2.0494],
         ...,
         [-2.0323, -1.8782, -1.8782,  ..., -2.0494, -2.0323, -2.0665],
         [-1.9638, -1.8268, -1.8782,  ..., -2.0665, -2.0665, -2.0837],
         [-2.0152, -1.9467, -1.9638,  ..., -2.0665, -2.0837, -2.0837]],

        [[-1.9657, -1.9657, -1.9657,  ..., -1.9657, -1.9657, -1.9657],
         [-1.9832, -1.9482, -1.9132,  ..., -1.9657, -1.9657, -1.9657],
         [-1.9657, -1.9132, -1.8957,  ..., -1.9657, -1.9657, -1.9657],
         ...,
         [-1.9482, -1.7906, -1.7906,  ..., -1.9657, -1.9482, -1.9832],
         [-1.8782, -1.7381, -1.7906,  ..., -1.9832, -1.9832, -2.0007],
         [-1.9307, -1.8606, -1.8782,  ..., -1.9832, -2.0007, -2.0007]],

        [[-1.7347, -1.7347, -1.7347,  ..., -1.7347, -1.7347, -1.7347],
         [-1.7522, -1.7173, -1.6824,  ..., -1

In [17]:
import torch.nn as nn
import torch.nn.functional as F

In [18]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, input_size = 12288):
        super(MyNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)
        pass

    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        pass

model = MyNeuralNetwork()
print(model)

MyNeuralNetwork(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)


In [19]:
loss_function = torch.nn.CrossEntropyLoss()
loss_function

CrossEntropyLoss()

In [20]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model.to(device)

MyNeuralNetwork(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [22]:
def train(start_epochs, n_epochs, model):
    for epoch in range(start_epochs, n_epochs + 1):
        print(f"epoch = {epoch}")

        pass

    # return trained model
    return model
    pass
train(0, 2, model)

epoch = 0
epoch = 1
epoch = 2


MyNeuralNetwork(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [23]:
def train(start_epochs, n_epochs, model):
    for epoch in range(start_epochs, n_epochs + 1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        #Set the model in training mode
        model.train()

        print(f"epoch = {epoch}")


    # return trained model
    return model
    pass
train(0, 2, model)

epoch = 0
epoch = 1
epoch = 2


MyNeuralNetwork(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [24]:
def train(start_epochs, n_epochs, model, train_loader):
    for epoch in range(start_epochs, n_epochs + 1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        #Set the model in training mode
        model.train()

        print(f"batch started: ")
        for batch_idx, (data, target) in enumerate(train_loader):
            #print(f"batch_idx: {batch_idx}")
            if batch_idx % 50 == 0:
                print(f"{batch_idx}, ", end = "")
            pass

        print(f"epoch = {epoch}")


    # return trained model
    return model
    pass
train(0, 2, model, train_data_loader)

batch started: 
0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, epoch = 0
batch started: 
0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, epoch = 1
batch started: 
0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, epoch = 2


MyNeuralNetwork(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [25]:
def train_process_batches(model, train_loader, optimizer, loss_function, verbose = True ):
    train_loss = 0.0

    model.train()
    if verbose:
        print(f"Training data batch process: ", end = "")

    for batch_idx, (data, target) in enumerate(train_loader):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()

        #we need to set the gradients to zero before starting to do backpropragation
        #because PyTorch accumulates the gradients on subsequent backward passes
        optimizer.zero_grad()

        #forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)

        #calculate the batch loss
        loss = loss_function(output, target)

        #backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()

        # perform a single optimization step (parameter update)
        optimizer.step()

        ## calculate train_loss
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        if batch_idx % 50 == 0:
            if verbose:
                print(f"\t{batch_idx}, {train_loss}", end = "\n")
            else:
                print(f"\t{batch_idx}, ", end = "")
        pass

    return train_loss
    pass

In [26]:
def train(start_epochs, n_epochs, model, train_loader):
    for epoch in range(start_epochs, n_epochs + 1):
        print(f"Epoch: {epoch}, ", end = "\n")
       # initialize variables to monitor training and validation loss
        valid_loss = 0.0

        #train model
        train_loss = train_process_batches(model, train_loader, optimizer, loss_function)

        print(f"\ntrain_loss = {train_loss}")
    # return trained model
    return model
train(0, 1, model, train_data_loader)

Epoch: 0, 
Training data batch process: 	0, 0.6143525242805481
	50, 0.012047462165355682
	100, 0.006083374377340078
	150, 0.0040690116584300995
	200, 0.003056820947676897
	250, 0.002447893377393484
	300, 0.002041266532614827
	350, 0.0017504877177998424
	400, 0.0015322223771363497
	450, 0.0013623526319861412
	500, 0.0012263888493180275
	550, 0.0011151012731716037
	600, 0.0010223309509456158
	650, 0.000943811028264463
	700, 0.0008764920639805496
	750, 0.000818137195892632
	800, 0.0007670675404369831
	850, 0.0007219990366138518
	900, 0.004823549184948206
	950, 0.0045699444599449635
	1000, 0.004341673571616411
	1050, 0.004135122988373041
	1100, 0.003947333432734013
	1150, 0.0037758606486022472
	1200, 0.0036186641082167625
	1250, 0.00347403297200799
	1300, 0.003340519033372402
	1350, 0.003216887591406703
	1400, 0.0031020808964967728
	1450, 0.0029951853211969137
	1500, 0.00289541226811707


IndexError: Target 2 is out of bounds.

In [27]:
def eval_process_batches(model, val_loader, optimizer, loss_function, verbose = True ):
    valid_loss = 0.0
    model.eval()
    if verbose:
        print(f"Test data batch process: ", end = "")

    for batch_idx, (data, target) in enumerate(val_loader):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        ## update the average validation loss
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = loss_function(output, target)
        # update average validation loss
        valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

        if batch_idx % 20 == 0:
            if verbose:
                print(f"\t{batch_idx}, {valid_loss}", end = "\n")
            else:
                print(f"\t{batch_idx}, ", end = "")
        pass
    return valid_loss
    pass

In [28]:
def train(start_epochs, n_epochs, model, train_loader, val_loader):
    for epoch in range(start_epochs, n_epochs+1):
        print(f"Epoch: {epoch}, ", end = "\n")
       # initialize variables to monitor training and validation loss
        valid_loss = 0.0

        #train model
        train_loss = train_process_batches(model, train_loader, optimizer, loss_function, verbose = False)
        valid_loss = eval_process_batches(model, val_loader, optimizer, loss_function, verbose = True)


        print(f"\ntrain_loss = {train_loss}")
        print(f"\nvalid_loss = {valid_loss}")

    # return trained model
    return model
train(0, 5, model, train_data_loader, val_data_loader)


Epoch: 0, 
	0, 	50, 	100, 	150, 	200, 	250, 	300, 	350, 	400, 	450, 	500, 	550, 	600, 	650, 	700, 	750, 	800, 	850, 	900, 	950, 	1000, 	1050, 	1100, 	1150, 	1200, 	1250, 	1300, 	1350, 	1400, 	1450, 	1500, 

IndexError: Target 2 is out of bounds.